# Process


### 1.Node Requirements/Create Company Node:
    a. Name(manual or from list)
        -Transform Name into SEC equivalent
    b. CIK(get_cik_by_company_name)
    c. (Optional) Set Status if error

    d. Create Company Node

### 2.Evaluate whether we initiate SEC call):
    a. Decide whether we search the SEC based on following:
        - Check if location is in US
        - Check if the company is the type of company that has 10-Ks
    b. (Optional) Set Status if rejected
    c. Test Name by seeing checking results of search
    d. (Optional) Set Status if rejected
    e. Check if CIK exists for company
        - CIK(get_cik_by_company_name)
        - Company Module in Edgar Library
    f. (Optional) Set Status if error
    g. Check if we can get 10-K
        - Verify Date using 10K 
            - Identify Date from first line
    h. (Optional) Set Status if error
    i. Attempt to pull EX 21.1 Document
        - Use name variations
    j. (Optional) Set Status if error


### 3.Parse Ex 21.1 Method in CompanyNode:
    a. Parse Columns, Tables of EX 21 into List
        - Get Names
        - Get Locations
    b. Create Dict of Name, CompanyNode(Name, Location, Parent) for each element in list 
    c. Repeat until program ends

## Testing:
#### -Accessing Same EX-21, Accessing Date of 10-K, Name Truncation, EX-21 not accessible

In [150]:
from edgar import Company, TXTML, Edgar
#test cases for 1) same document, 2) find date of 10-K 3) different name variations 4) exhibit pull fails (multiple names + not in seperate file)




#TEST CASE 1:


#same document: Berkshire Hathaway Nevada Power and Pacificorp

# possible_companies = edgar.find_company_name("PACIFICORP")
# print(possible_companies)

# cik = edgar.get_cik_by_company_name('PACIFICORP /OR/')
# #print(cik)

PacifiCorp = Company("PACIFICORP","0000075594")


#RESULTS 1:
#edgar.get_cik_by_company_name needs to have exact name as company(in SEC) or it won't work. Will throw error if not exact or doesn't exist
#sub-case: test with string that accepts only one result 




#TEST CASE 2:

#Find date of 10-K to confirm if usable

#get 10-K to check date
# paci_10K = PacifiCorp.get_10K()
# print(paci_10K[0])

#Doesn't Work:
#html = TXTML.get_HTML_from_document(paci_10K[0]) 
#print(html)

#Works:
# doc = TXTML.get_document_type(paci_10K[0]) #returns document with promising date in first line
# print(doc)


#RESULTS 2:
#first line: 20190222182150
#YYYY-MM-DD
#can use first line of TXTML.get_document_type function to get date of 10-K, letting us know if its in a range we can use





#paci_21 = PacifiCorp.get_document_type_from_10K("EX-21.1")
#doc = TXTML.get_document_type(paci_21[0]) #returns document
#paci_text = TXTML.get_HTML_from_document(paci_21[0])
#print(doc)


# possible_companies = edgar.find_company_name("NEVADA POWER")
# print(possible_companies)

# cik = edgar.get_cik_by_company_name('NEVADA POWER CO')
# print(cik)

Nevada = Company("NEVADA POWER CO","0000071180")

neva_21 = Nevada.get_document_type_from_10K("EX-21.1")
neva_text = TXTML.get_HTML_from_document(neva_21[0])
# print(neva_text)


#TEST CASE 3

#different name variations 
'''
Types of companies - Result - Notes - Link

Group
Synchronous Aerospace Group - X


Company
Ingersoll Cutting Tool Company - X
Kern River Gas Transmission Company - X
National Indemnity Company - NATIONAL INDEMNITY CO - (No 10-K)

Corporation
Ben Bridge Corporation - X
Special Metals Corporation - SPECIAL METALS CORP - 21.1 nested subsidiaries (https://www.sec.gov/Archives/edgar/data/1028965/000095012302003300/y58954ex21-1.txt
)
The Lubrizol Corporation - LUBRIZOL Corp - 21.1 Location Naming Convention: USA - Delaware (https://www.sec.gov/Archives/edgar/data/60751/000119312511046757/dex211.htm)
Titanium Metals Corporation - TITANIUM METALS CORP - 21.1 -(https://www.sec.gov/Archives/edgar/data/1011657/000119312512087802/d228252dex211.htm)
XTRA Corporation - XTRA CORP /DE/ - 21 - no tables, just tabs (https://www.sec.gov/Archives/edgar/data/217591/000095013100006859/0000950131-00-006859-0010.txt)


Ltd.
The Pampered Chef, Ltd. - X


LLC
Cerro Wire LLC - X
Charter Brokerage LLC - X
BH Finance LLC - BH Finance LLC - (No 10-K)
Burlington Northern Santa Fe, LLC - BURLINGTON NORTHERN SANTA FE, LLC 10-K, no EX-21 (https://www.sec.gov/Archives/edgar/data/934612/000093461219000005/0000934612-19-000005-index.htm)
Bushwick Metals LLC - X
Campbell Hausfeld, LLC - X
Duracell U.S. Holding LLC - X

Inc
BH Media Group, Inc. - X
Empire Distributors, Inc. - X
Fruit of the Loom, Inc. - FRUIT OF THE LOOM LTD - 21 - (https://www.sec.gov/Archives/edgar/data/1053303/000095013702002190/c68623ex21.txt)
International Dairy Queen, Inc. - INTERNATIONAL DAIRY QUEEN INC(No period) - 21 - but not in document path (https://www.sec.gov/Archives/edgar/data/51207/0000912057-95-000603-index.html)
M&M Manufacturing, Inc. - X
MHC Inc. - X


Incorporated
Garan, Incorporated - GARAN INC (no space, might not be a match) - 21 - (https://www.sec.gov/Archives/edgar/data/39917/000103352501500037/ex21.txt)
Progressive Incorporated - X

Holdings
GRD Holdings Corporation - X
Railsplitter Holdings Corporation - X
Henley Holdings, LLC X
NVE Holdings, LLC X
Marmon Holdings, Inc. - MARMON HOLDINGS INC (No 10-K)
OTC Worldwide Holdings, Inc. X

JV

Combos
L. A. Darling Company LLC X
Marmon Retail & Highway Technologies Company LLC X
Union Underwear Company, Inc. - UNION UNDERWEAR CO INC - (No 10-K)                         
Applied Underwriters Captive Risk Assurance Company, Inc. X
H. H. Brown Shoe Company, Inc. X
Söfft Shoe Company, LLC X 

'''

# RESULTS 3:
'''
-No punctuation
-Abbreviation
-If "The" is the first word, remove it
-Corporation gets shortened to Corp
-Company gets shortened to Co
-LLCS can go as is even with punctuation
-Incs don't need a period
-Occasional /##/ in corp name: ex. PACIFICORP /OR/ , XTRA CORP /DE/ (looks like it could be the date of incorporation but not the current location, might be able to get from 10-K)
'''


#TEST CASE 4


#Accessing EX-21 when it is not provided as a link in 10-K

#Example: https://www.sec.gov/Archives/edgar/data/51207/0000912057-95-000603-index.html

# test_comp = Company('INTERNATIONAL DAIRY QUEEN INC', '0000051207')
# test_21 = test_comp.get_document_type_from_10K('EX-21')
# test_text = TXTML.get_HTML_from_document(test_21[2])
# print(test_text)


#RESULTS:
#It looks like we can pull in the document even if the link isn't there (we get a list of 3 that's labeled an element_html)
#Not able to use get_HTML_from_document on it unlike one that works. Might be because it is taking in text rather than link

'\n-No punctuation\n-Abbreviation\n-If "The" is the first word, remove it\n-Corporation gets shortened to Corp\n-Company gets shortened to Co\n-LLCS can go as is even with punctuation\n-Incs don\'t need a period\n-Occasional /##/ in corp name: ex. PACIFICORP /OR/ , XTRA CORP /DE/ (looks like it could be the date of incorporation but not the current location, might be able to get from 10-K)\n'

In [ ]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

class CompanyNode:
    def __init__(self, name, location, parent = None):
        self.name = name
        
        #tries to find CIK on SEC website
        try:
            self.cik = edgar.get_cik_by_company_name(self.name)
            self.SEC = True
        except:
            self.cik = Noneself.sic = None        
        
        self.location = location

        self.corp_type = eval_corp_type()
        
        self.parent = parent
        self.child = {}
        
        
        
        self.SEC = eval_SEC() #indicates whether the company can be found on SEC
        

    def eval_corp_type(self):
        corp_check = self.name.split()
        print(corp_check[-1])
        return corp_check[-1]
    
    def set_SIC(self, SIC):
        self.sic = SIC
    
    def eval_SEC(self):
        if self.location in states and cik != None:
            self.SEC = True
        else:
            self.SEC = False
    
    def set_child



- SIC()
- Corp Type (Found through subsidiary document of parent)
- Location(Found through subsidiary document of parent)
- Parent (Found through subsidiary document of parent)
- Child (found through subsidiary document of self)

## Junk code below, ignore:

In [ ]:
import edgar
import inspect
import random
dir(edgar)
company2 = edgar.Company("IAC/INTERACTIVECORP", "0000891103")

In [ ]:
from edgar import Company, TXTML, Edgar



#https://www.sec.gov/Archives/edgar/data/891103/000089110319000006/0000891103-19-000006-index.htm


company = Company("IAC/INTERACTIVECORP", "0000891103")
doc = company.get_10K()
# doc_type = TXTML.get_document_type(doc[0]) #returns document
# print(doc_type) #first line: 20190301171016
#YYYY-MM-DD


#doc_text = TXTML.get_HTML_from_document(doc)
#print(doc_text)
#text = TXTML.parse_full_10K(doc)

text = company._get_filings_url()
#https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000891103&type=&dateb=&owner=include&count=100



#text = company._group_document_type()
test21 = company.get_document_type_from_10K("EX-21.1")
#text = test21[0]._get_filings_url()
#print(test21)
#dir(TXTML)
#text = TXTML.get_document_type(test21[0])


text = TXTML.get_HTML_from_document(test21[0])
# iac-ex211_20181231.htm


#text = TXTML.parse_full_10K(test21[0])

#print(test21)
#print(text)
dir(TXTML)
#dir(Company)
#dir(Edgar)

#edgar = Edgar()
possible_companies = edgar.find_company_name("XTRA CORP")
cik = edgar.get_cik_by_company_name('XTRA CORP')

print(cik)